In [1]:
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from collections import Counter
from pymystem3 import Mystem
from classification_model import train_test_datasets, NeuralNetwork, train_nn, predict, multilabel_accuracy, get_test_loader
from data_preprocessing import clean_data, apply_n_grams, vectorize_text
import re
import math
import joblib
import pickle
import numpy as np
from gensim.models import Word2Vec
import torch
import torch.nn as nn

In [2]:
tproger = pd.read_json('/Users/alexandrasorokovikova/Data Science/НИР Сравнение IT/Comparison-of-IT-trends/data/tproger.json')
tproger


,time,tags,views,comments,saved
0,2021-11-07T13:13:54+03:00,"[Интернет, Карьера, Работа, Сервисы]",5080,2,10
1,2021-11-03T14:39:24+03:00,"[Алгоритмы, Математика и теория вероятностей]",1779,0,4
2,2021-11-01T13:54:57+03:00,[Пост пользователя],380,0,0
3,2021-11-11T13:10:01+03:00,[Партнёрский материал],1222,0,5
4,2021-11-10T19:09:24+03:00,"[JavaScript, Алгоритмы, Для начинающих, Пост п...",562,0,2
...,...,...,...,...,...
2980,2014-12-13T00:20:42+03:00,[Git],208,0,0
2981,2013-12-01T23:55:57+04:00,[],5106,5,0
2982,2013-03-08T16:24:40+04:00,"[Unity, Для продвинутых, Для продолжающих, Раз...",10932,0,0
2983,2014-12-05T12:11:51+03:00,[Интернет],1767,0,1


In [3]:
fields = {0 : 'Железо, электроника',
         1 : 'Сетевые технологии, администрирование, devops',
         2 : 'Проектирование, тестирование',
         3 : 'Информационная безопасность',
         4 : 'Маркетинг, контекстная реклама',
         5 : 'Верстка, html, javascript',
         6 : 'Машинное обучение',
         7 : 'E-commerce, финансы',
         8 : 'Дизайн интерфейсов',
         9 : 'Бизнес, CRM',
         10 : 'SEO'}

In [4]:
word2vec_model = Word2Vec.load('habr_vocab_model_on_text')

## Подготовка признаков

In [5]:
def join(row):
    if len(row) != 0:
        return ' '.join(row)
tproger['tags'] = tproger['tags'].apply(join)
tproger = tproger.dropna()
tproger.head()

,time,tags,views,comments,saved
0,2021-11-07T13:13:54+03:00,Интернет Карьера Работа Сервисы,5080,2,10
1,2021-11-03T14:39:24+03:00,Алгоритмы Математика и теория вероятностей,1779,0,4
2,2021-11-01T13:54:57+03:00,Пост пользователя,380,0,0
3,2021-11-11T13:10:01+03:00,Партнёрский материал,1222,0,5
4,2021-11-10T19:09:24+03:00,JavaScript Алгоритмы Для начинающих Пост польз...,562,0,2


In [6]:
tproger['tags'] = clean_data(tproger['tags'])
tproger.head()

100%|██████████| 2842/2842 [00:02<00:00, 1073.18it/s]
/Users/alexandrasorokovikova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,time,tags,views,comments,saved
0,2021-11-07T13:13:54+03:00,интернет карьера работа сервис,5080,2,10
1,2021-11-03T14:39:24+03:00,алгоритм математика теория вероятность,1779,0,4
2,2021-11-01T13:54:57+03:00,пост пользователь,380,0,0
3,2021-11-11T13:10:01+03:00,партнерский материал,1222,0,5
4,2021-11-10T19:09:24+03:00,javascript алгоритм начинающий пост пользователь,562,0,2


In [7]:
tproger = tproger[(tproger['tags'] != 'пост пользователь') & (tproger['tags'] != 'партнерский материал') & (tproger['tags'] != 'начинающий')]
tproger.index = np.arange(len(tproger))
tproger_train = tproger[['tags']]


In [9]:
tproger_train['text_n_grams'] = tproger_train['tags'].apply(apply_n_grams, word2vec_model=word2vec_model)
tproger_train.head()

/Users/alexandrasorokovikova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tags,text_n_grams
0,интернет карьера работа сервис,"[интернет, карьера, работа, сервис]"
1,алгоритм математика теория вероятность,"[алгоритм, математика, теория, вероятность]"
2,javascript алгоритм начинающий пост пользователь,"[javascript, алгоритм, начинающий, пост, польз..."
3,devops пост пользователь,"[devops, пост, пользователь]"
4,аналитик управление проект,"[аналитик, управление, проект]"


In [10]:
tproger_train['vector'] = tproger_train['text_n_grams'].apply(vectorize_text, word2vec_model=word2vec_model)
tproger_train.head()

/Users/alexandrasorokovikova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tags,text_n_grams,vector
0,интернет карьера работа сервис,"[интернет, карьера, работа, сервис]","[0.9599026720142433, -2.386348231296942, 1.925..."
1,алгоритм математика теория вероятность,"[алгоритм, математика, теория, вероятность]","[-0.4488298760784523, -1.5717914208587118, 1.5..."
2,javascript алгоритм начинающий пост пользователь,"[javascript, алгоритм, начинающий, пост, польз...","[0.05509903509545582, -2.0339352249100644, 1.3..."
3,devops пост пользователь,"[devops, пост, пользователь]","[0.6361756465597285, -1.0281147665779689, 2.23..."
4,аналитик управление проект,"[аналитик, управление, проект]","[1.7125937975083068, -1.833044327334939, 1.954..."


In [11]:
tproger_train[list(fields.values())] = 0
tproger_train.head()

/Users/alexandrasorokovikova/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


,tags,text_n_grams,vector,"Железо, электроника","Сетевые технологии, администрирование, devops","Проектирование, тестирование",Информационная безопасность,"Маркетинг, контекстная реклама","Верстка, html, javascript",Машинное обучение,"E-commerce, финансы",Дизайн интерфейсов,"Бизнес, CRM",SEO
0,интернет карьера работа сервис,"[интернет, карьера, работа, сервис]","[0.9599026720142433, -2.386348231296942, 1.925...",0,0,0,0,0,0,0,0,0,0,0
1,алгоритм математика теория вероятность,"[алгоритм, математика, теория, вероятность]","[-0.4488298760784523, -1.5717914208587118, 1.5...",0,0,0,0,0,0,0,0,0,0,0
2,javascript алгоритм начинающий пост пользователь,"[javascript, алгоритм, начинающий, пост, польз...","[0.05509903509545582, -2.0339352249100644, 1.3...",0,0,0,0,0,0,0,0,0,0,0
3,devops пост пользователь,"[devops, пост, пользователь]","[0.6361756465597285, -1.0281147665779689, 2.23...",0,0,0,0,0,0,0,0,0,0,0
4,аналитик управление проект,"[аналитик, управление, проект]","[1.7125937975083068, -1.833044327334939, 1.954...",0,0,0,0,0,0,0,0,0,0,0


## Поиск категорий

In [12]:
target_list = list(fields.values())
test_loader = get_test_loader(tproger_train, target_list)

In [13]:
model = NeuralNetwork()
model.load_state_dict(torch.load("nn_classifier"))
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): Tanh()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Tanh()
    (4): Linear(in_features=512, out_features=11, bias=True)
  )
)

In [14]:
predictions = predict(model, test_loader)

In [15]:
def get_field(row):
    all_fields = []
    for i in row:
        all_fields.append(fields[i])
    return all_fields

tproger['field'] = predictions
tproger['field'] = tproger['field'].apply(get_field)
tproger

,time,tags,views,comments,saved,field
0,2021-11-07T13:13:54+03:00,интернет карьера работа сервис,5080,2,10,"[Бизнес, CRM, Железо, электроника, Проектирова..."
1,2021-11-03T14:39:24+03:00,алгоритм математика теория вероятность,1779,0,4,"[Железо, электроника, Проектирование, тестиров..."
2,2021-11-10T19:09:24+03:00,javascript алгоритм начинающий пост пользователь,562,0,2,"[Проектирование, тестирование, Верстка, html, ..."
3,2021-11-08T13:46:06+03:00,devops пост пользователь,687,1,0,"[Сетевые технологии, администрирование, devops..."
4,2021-10-29T13:35:38+03:00,аналитик управление проект,5831,2,8,"[Бизнес, CRM]"
...,...,...,...,...,...,...
2801,2014-12-21T01:26:51+03:00,linux,4141,0,0,"[Сетевые технологии, администрирование, devops..."
2802,2014-12-15T19:10:37+03:00,google головоломка задача повышенный сложность,423904,8,4,"[Железо, электроника, Сетевые технологии, адми..."
2803,2014-12-13T00:20:42+03:00,git,208,0,0,"[Проектирование, тестирование]"
2804,2013-03-08T16:24:40+04:00,unity продвинутый продолжать разработка игра,10932,0,0,"[Верстка, html, javascript, Железо, электроник..."


In [ ]:
#tproger.to_json('tproger_final.json')